### 3.6 Softmax回归的从零开始实现

In [15]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import sys
import os, sys
sys.path.append("..")
import d2lzh_pytorch.utils as d2l
np.random.seed(666)
cur_path = os.path.abspath(os.path.dirname('__file__'))
data_path = cur_path.replace('dl\dive-into-dl\chapter03-dl-basics', 'data\\')
data_path

'D:\\develop\\workspace\\pycharm\\machine-learning\\data\\'

### 3.6.1 获取和读取数据

In [17]:
batch_size = 256
if sys.platform.startswith('win'):
    cur_path = os.path.abspath(os.path.dirname('__file__'))
    data_path = cur_path.replace('dl\dive-into-dl\chapter03-dl-basics', 'data\\')
    train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, root=data_path)
else:
    train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

### 3.6.2 初始化模型参数

In [18]:
num_inputs = 784
num_outputs = 10

W = torch.tensor(np.random.normal(0, 0.01, (num_inputs, num_outputs)), dtype=torch.float, requires_grad=True)
b = torch.zeros(num_inputs, num_outputs, dtype=torch.float, requires_grad=True)

In [19]:
X = torch.tensor([[1, 2, 3], [4, 5, 6]])
# 按行或者按列求和，并保持维度
print(X.sum(dim=0, keepdim=True))
print(X.sum(dim=1, keepdim=True))

tensor([[5, 7, 9]])
tensor([[ 6],
        [15]])


### 3.6.3 实现softmax运算

In [20]:
def softmax(X):
    # 求每个元素的exp
    X_exp = X.exp()
    partition = X_exp.sum(dim=1, keepdim=True)
    return X_exp / partition

### 3.6.4 定义模型

In [28]:
def net(X):
    return softmax(torch.mm(X.view((1, num_outputs)), W) + b)

### 3.6.5 定义损失函数

In [29]:
# out[i][j][k] = input[index[i][j][k]][j][k]  # dim=0
# out[i][j][k] = input[i][index[i][j][k]][k]  # dim=1
# out[i][j][k] = input[i][j][index[i][j][k]]  # dim=2

y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y = torch.LongTensor([0, 2])
y_hat.gather(1, y.view(-1, 1))


tensor([[0.1000],
        [0.5000]])

In [30]:
def cross_entropy(y_hat, y):
    return - torch.log(y_hat.gather(1, y.view(-1, )))

### 3.6.6 计算分类准确率

In [31]:
def accuracy(y_hat, y):
    return (y_hat.argmax(dim=1) == y).float().mean().item()

In [32]:
print(accuracy(y_hat, y))

0.5


In [33]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

In [34]:
print(evaluate_accuracy(test_iter, net))

RuntimeError: shape '[1, 10]' is invalid for input of size 200704